###### Webscrape_v5 - borrowed from v5 indent error problem in myspider
https://stackoverflow.com/questions/57616611/python-scrapy-issues-running-imagespipeline-when-running-crawler-from-scrip

In [1]:
import scrapy
import logging
import json
from scrapy.crawler import CrawlerProcess
import pandas as pd
from pandas import read_csv
import csv
from scrapy.shell import inspect_response
import numpy as np
from scrapy.pipelines.images import ImagesPipeline
from scrapy.selector import Selector

In [2]:
class JsonWriterPipeline(object):

# in many examples the Pipeline is defined as an object, e.g.
# class JsonWriterPipeline(object):
# strangely this seems to be equivalent to the above, i.e. leaving
# out the "object" argument.
    
    def open_spider(self,spider):
        self.file = open('item.jl', 'w')

    def close_spider(self,spider):
        self.file.close()

    def process_item(self, item,spider):
        print('hello world',dict(item))
        for field in item:
            print ('Json items',field + ': ' + item[field])
            
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
class BasicPipeline(object):
    
    def open_spider(self,spider):
        self.file = open('test2.txt','w')
        
    def close_spider(self,spider):
        self.file.close()
        
    def process_item(self,item,spider):
        print("basic")
        line=str(item)
        self.file.write(line + "\n")
        return item

In [4]:
class MyNewPipeline(object):
    
    def open_spider(self,spider):
        self.file = open('items2.jl', 'wb')
        
    def close_spider(self, spider):
        self.file.close()
        
    def process_item(self,item,spider):
        print('processing',dict(item))
        line = json.dumps(dict(item)) + " new " + "\n"
        self.file.write(line)
        return item

In [5]:
class ItemCollectorPipeline(object):
    def __init__(self):
        self.ids_seen = set()
        
    def process_item(self,item,spider):
        print("ItemCollector")
        items.append(item)

In [6]:
class ImageItem(scrapy.Item):
    images = scrapy.Field()
    image_urls = scrapy.Field()

In [7]:
class ItemTest(scrapy.Item):
    Organization = scrapy.Field()
    url = scrapy.Field()
    robot = scrapy.Field()
    machine = scrapy.Field()
    tinyML = scrapy.Field()
    brain = scrapy.Field()
    food = scrapy.Field()
    textout = scrapy.Field()

In [8]:
class Scrapy2Item(scrapy.Item):
    title = scrapy.Field()
    image_urls = scrapy.Field()
    sku = scrapy.Field()

class Scrapy2Pipeline(ImagesPipeline):
    def get_media_requests(self, item, info):
        return [scrapy.Request(x, meta={'image_name': item['sku']})
                for x in item.get('image_urls', [])]

    def file_path(self, request, response=None, info=None):
        return '%s.jpg' % request.meta['image_name']

In [9]:
class ImgSpider(scrapy.Spider):
    name = "img_spider"
    start_urls = ["https://www.amazon.ca/s?k=821826022317"]
#    domain = "https://www.amazon.ca/s?k=821826022317"
#    print('domain',domain)
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES':{'scrapy.pipelines.images.ImagesPipeline':1,
                         '__main__.ItemCollectorPipeline':100}
#        'DOWNLOAD_TIMEOUT':{1200}
    }
    
#    def start_requests(self):
#        yield scrapy.Request(url=ImgSpider.domain, callback = self.parse)
    
    def parse(self, response):
        image = ImageItem()
        img_urls = []
        items = Scrapy2Item()
        
        titlevar = response.css('span.a-text-normal ::text').extract_first()
        imgvar = [response.css('img ::attr(src)').extract_first()]
        print('titlevar',titlevar)
        
        
        for img in response.css("img::attr(src)").extract():
            print("img",img)
            img_urls.append(img)

        items['title'] = titlevar
        items['image_urls'] = imgvar
        print('items',items)
            
        image["image_urls"] = img_urls
        print("image",image)
        
        yield items

In [10]:
class MySpider(scrapy.Spider):
    name = "startups"
    
# read crunchbase file and pick off websites
    dataset1=read_csv('C:/Users/MStopa/WebScraping&Crunchbase/WebScraping&Crunchbase/Crunchbase/AI-6-25-2021.csv')
    df=dataset1[['Organization Name','Full Description','Headquarters Location','Website']]
    start_urlsA=df['Website'].to_numpy().tolist()
    start_urlsB=start_urlsA[:950]
    start_urls = [x for x in start_urlsB if str(x) != 'nan']
    companies=df['Organization Name'].to_numpy().tolist()
    print('start_urls here',start_urls)
#    print('companies[0]',companies[0])

    
    custom_settings = {
        'LOG_LEVEL': logging.DEBUG,
        'ITEM_PIPELINES':{'__main__.ItemCollectorPipeline':10},
        'ITEM_PIPELINES':{'scrapy.pipelines.images.ImagesPipeline':100},
        'ITEM_PIPELINES':{'MyNewPipeline':6},
        'ITEM_PIPELINES':{'__main__.JsonWriterPipeline':12},
        'ITEM_PIPELINES':{'__main__.BasicPipeline':4},
    }
    
    count=0
    url_list=[]
    def parse(self, response):
        sitep=response.xpath('//p/text()').getall()
        badchars = ["\U0001f48c","\U0001f4f1","\u2011","\u301d","\ufb01","\u2023","\u011f","\u1ea3","\U0001f973",\
                    "\U0001f973","\U0001f4da","\U0001f33f","\u2192","\u200d","\u2028","\u24d2","\u2011",\
                   "\u010d","\U0001f64c","\u200b","\n","\t","\r"]
        bcnum=len(badchars)
        for idx,string1 in enumerate(sitep):
            for jj in range(bcnum):
                string1 = string1.replace(badchars[jj],'')

            string1 = string1.replace(u'xa0',u' ')

            if string1:
                if string1[-1] != '.':
                    string1 += '.'
                    
                    
            sitep[idx]=string1
                    
        out1=' '.join(sitep)
        site_url=response.url
        print('\n out1',out1[0:100])
        if out1:
            out1=site_url + ' ' + out1
            file1=open("finance.cor",'a')
            out1=out1.lstrip(' ')
            file1.write(out1 + '\n')
        items=ItemTest()

        with open('test1.txt','a') as page:
            page.write(str(sitep))
            page.close()

        subs='tinyML'
        result1 = [i for i in sitep if subs in i]
        if result1: # boolean - list is not empty
            self.url_list.append(response.url)
            items['tinyML']=response.xpath('//title/text()').get()
            items['url']=response.url
            items['textout']=response.xpath('//p/text()').getall()
            yield items

        subs2='brain'
        result1 = [i for i in sitep if subs2 in i]
        if result1: # boolean - list is not empty
            self.url_list.append(response.url)
            items['brain']=response.xpath('//title/text()').get()
            items['url']=response.url
            items['textout']=response.xpath('//p/text()').getall()
            yield items

        subs3='food'
        result1 = [i for i in sitep if subs3 in i]
        if result1: # boolean - list is not empty
            self.url_list.append(response.url)
            items['food']=response.xpath('//title/text()').get()
            items['url']=response.url
            items['textout']=response.xpath('//p/text()').getall()
            yield items
            
        subs4='robot'
        result1 = [i for i in sitep if subs4 in i]
        if result1: # boolean - list is not empty
            self.url_list.append(response.url)
            
            items['url']=response.url
            items['textout']=response.xpath('//p/text()').getall()
            yield items
            
        print('this url',response.url)
        print('companies[0]',self.companies[self.count])
        self.count+=1

# print('\n url',subs2,response.url)


start_urls here ['http://eightfold.ai', 'https://www.openai.com/', 'http://sift.com', 'http://plus.ai/', 'https://www.deepmap.ai/', 'https://salt.security/', 'https://commerceiq.ai', 'http://relativityspace.com', 'https://grabango.com', 'https://primer.ai', 'http://alation.com', 'http://harness.io/', 'https://userleap.com', 'https://www.vian.ai/', 'https://vectra.ai/', 'https://cowbell.insure', 'https://www.mythic-ai.com/', 'http://determined.ai', 'http://www.upstart.com', 'https://getjerry.com/', 'https://www.skydio.com', 'http://www.chorus.ai', 'https://people.ai/', 'https://tonkean.com', 'http://www.clari.com', 'http://h2o.ai/', 'http://www.eltropy.com', 'https://www.dremio.com/', 'https://www.aifi.com', 'https://crate.io', 'http://www.feedzai.com', 'http://www.tonal.com', 'https://www.brightmachines.com', 'https://www.getmagic.com/', 'https://www.yalochat.com/', 'https://www.agari.com', 'https://www.forethought.ai', 'http://www.oneconcern.com/', 'https://observe.ai/', 'http://komod

In [11]:
outVar='outside_class'
IMAGES_STORE='C://MStopa/WebScraping/images'
items=[]
process = CrawlerProcess({
    'ITEM_PIPELINES':{'MyNewPipeline':2},
#    "FEEDS": {"items.json": {"format": "json"}},
#    "LOG_FILE": 'scrapy_log.txt',
    "DOWNLOAD_DELAY":5
})


process.crawl(MySpider)
# process.crawl(ImgSpider)
process.start() # the script will block here until the crawling is finished
# print("image",image)
# print(image)
for item in items:
    print('item',item)

2021-07-15 21:06:25 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: scrapybot)
2021-07-15 21:06:25 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 2.9.2, Platform Windows-10-10.0.19041-SP0
2021-07-15 21:06:25 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-07-15 21:06:25 [scrapy.crawler] INFO: Overridden settings:
{'DOWNLOAD_DELAY': 5, 'LOG_LEVEL': 10}
2021-07-15 21:06:26 [scrapy.extensions.telnet] INFO: Telnet Password: af9935c10971e56a
2021-07-15 21:06:26 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-07-15 21:06:28 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.d


 out1 In our rapidly shifting world of digital transformation, API-first strategies are a priority. The se
this url https://salt.security/
companies[0] Eightfold

 out1 Imagine there were no checkout lines and you could use a store like it was your personal pantry. Gra
this url https://grabango.com
companies[0] OpenAI

 out1 We’ve developed Codex, an AI that translates natural language into code. Together with GitHub we’re 
this url https://www.openai.com/
companies[0] Sift

 out1 
this url https://primer.ai
companies[0] Plus


2021-07-15 21:06:29 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.upstart.com/> from <GET http://www.upstart.com>
2021-07-15 21:06:29 [scrapy.core.engine] DEBUG: Crawled (403) <GET https://cowbell.insure> (referer: None)
2021-07-15 21:06:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.skydio.com> (referer: None)
2021-07-15 21:06:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://getjerry.com/> (referer: None)
2021-07-15 21:06:29 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.vectra.ai/> from <GET https://vectra.ai/>
2021-07-15 21:06:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.vian.ai/> (referer: None)
2021-07-15 21:06:29 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <403 https://cowbell.insure>: HTTP status code is not handled or not allowed



 out1 Enhancing Mapping Capabilities for the Autonomous Vehicle Industry. DeepMap provides the world’s bes
this url https://www.deepmap.ai/
companies[0] DeepMap


2021-07-15 21:06:29 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.chorus.ai/> from <GET http://www.chorus.ai>
2021-07-15 21:06:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://userleap.com> (referer: None)
2021-07-15 21:06:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://people.ai/> (referer: None)
2021-07-15 21:06:29 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.clari.com/> from <GET http://www.clari.com>
2021-07-15 21:06:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mythic-ai.com/> (referer: None)


...

2021-07-15 21:12:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.icims.com> (referer: None)



 out1 Learn key trends and areas of opportunity in this exclusive report from iCIMS and Talent Board. Lear
this url https://www.icims.com
companies[0] Wit.ai


2021-07-15 21:12:40 [scrapy.downloadermiddlewares.retry] ERROR: Gave up retrying <GET https://www.gild.com> (failed 3 times): TCP connection timed out: 10060: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond..
2021-07-15 21:12:40 [scrapy.core.scraper] ERROR: Error downloading <GET https://www.gild.com>
Traceback (most recent call last):
  File "C:\Users\MStopa\AppData\Local\Continuum\anaconda3\envs\webscrape\lib\site-packages\scrapy\core\downloader\middleware.py", line 44, in process_request
    return (yield download_func(request=request, spider=spider))
twisted.internet.error.TCPTimedOutError: TCP connection timed out: 10060: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond..
2021-07-15 21:12:45 [scrapy.downloadermiddlew


 out1 
this url http://www.downtownapp.co
companies[0] MD Insider


2021-07-15 21:13:27 [scrapy.downloadermiddlewares.retry] ERROR: Gave up retrying <GET http://quantiply.com> (failed 3 times): TCP connection timed out: 10060: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond..
2021-07-15 21:13:28 [scrapy.core.scraper] ERROR: Error downloading <GET http://quantiply.com>
Traceback (most recent call last):
  File "C:\Users\MStopa\AppData\Local\Continuum\anaconda3\envs\webscrape\lib\site-packages\scrapy\core\downloader\middleware.py", line 44, in process_request
    return (yield download_func(request=request, spider=spider))
twisted.internet.error.TCPTimedOutError: TCP connection timed out: 10060: A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond..
2021-07-15 21:13:28 [scrapy.extensions.logsta